In [1]:
import asyncio
import websockets
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from dash import Dash, dcc, html
from dash.dependencies import Input, Output
import threading
import webbrowser
from flask import Flask

# generate sample timestamp


# Generate sample data
data = np.append(
            np.append(
                np.append(
                    np.random.randint(50,61,(100,1)),
                    np.random.randint(0,100,(100,1)), axis=1),
                np.random.randint(-100,0,(100,1)), axis=1),
            np.random.randint(1,3,(100,1)), axis=1)

df = pd.DataFrame(data, columns=["timeStamp", "TxPower","RSSI","RTT"])
df["timeStamp"] = df["timeStamp"].cumsum(axis=0)

In [35]:
fig = go.Figure()

power_color = '#636EFA'
rssi_color = '#EF553B'
rtt_color = '#00CC96'
grid_color = "rgb(75,75,75)"

fig.update_layout(font_color="rgb(150,150,150)")

fig.add_trace(go.Scatter(x=df["timeStamp"], y=df["TxPower"], name="TxPower", mode="lines", line=dict(color=power_color)))
fig.add_trace(go.Scatter(x=df["timeStamp"], y=df["RSSI"], name="RSSI", mode="lines", line=dict(color=rssi_color), yaxis="y2"))
fig.add_trace(go.Scatter(x=df["timeStamp"], y=df["RTT"], name="RTT", mode="lines", line=dict(color=rtt_color), yaxis="y3"))

fig.update_layout(
                  title=dict(text="Radio Transmission metrics", font_size=24),
                  width=1750,
                  height=800,
                  legend=dict(x=0, y=1.07, orientation='h'),
                  plot_bgcolor='rgb(35,35,35)',
                  paper_bgcolor ='rgb(10,10,10)',
                  xaxis=dict(domain=[0.05,1], linecolor=grid_color, gridcolor=grid_color, mirror=True),
                  yaxis=dict(title=dict(text="TxPower (dBm)", font=dict(color=power_color)), tickfont=dict(color=power_color), range=[0,100], linecolor=grid_color, gridcolor=grid_color),
                  yaxis2=dict(title=dict(text="RSSI (dBm)", font=dict(color=rssi_color)), tickfont=dict(color=rssi_color), anchor="free", overlaying="y", side="left", position=0, range=[-100,0], linecolor=grid_color, gridcolor=grid_color),
                  yaxis3=dict(title=dict(text="RTT (ms)", font=dict(color=rtt_color)), tickfont=dict(color=rtt_color), anchor="x", overlaying="y", side="right", range=[0,5], linecolor=grid_color, gridcolor=grid_color)
                )

fig.show()

In [ ]:
import pandas as pd
import scipy.stats as stats
import plotly.express as px
import plotly.graph_objects as go

# Load your data (replace 'your_data.csv' with your actual data source)
df = pd.read_csv('your_data.csv')

# Function to check assumptions within each power group
def check_assumptions(data, group_col, value_col, power_col):
    power_groups = data[power_col].unique()

    for power in power_groups:
        subset = data[data[power_col] == power]
        groups = subset[group_col].unique()

        print(f"\nChecking assumptions for power group {power}:")

        # Normality check using Shapiro-Wilk test
        print("\nNormality Check (Shapiro-Wilk Test):")
        for group in groups:
            group_data = subset[subset[group_col] == group][value_col]
            stat, p_value = stats.shapiro(group_data)
            print(f'{group}: W={stat:.4f}, p-value={p_value:.4f}')

            # Q-Q Plot for each group using Plotly
            qq_fig = go.Figure()
            qq_fig.add_trace(go.Scatter(
                x=stats.probplot(group_data, dist="norm")[0][0], 
                y=stats.probplot(group_data, dist="norm")[0][1],
                mode='markers',
                name=f'{group} Q-Q Plot'
            ))
            qq_fig.add_trace(go.Scatter(
                x=stats.probplot(group_data, dist="norm")[0][0], 
                y=stats.probplot(group_data, dist="norm")[0][0],
                mode='lines',
                name='Ideal Line'
            ))
            qq_fig.update_layout(
                title=f'Q-Q Plot for {group} (Power Group {power})',
                xaxis_title='Theoretical Quantiles',
                yaxis_title='Sample Quantiles',
                template='plotly_dark'
            )
            qq_fig.show()

        # Homogeneity of variances check using Levene's test
        print("\nHomogeneity of Variances Check (Levene's Test):")
        group_data = [subset[subset[group_col] == group][value_col] for group in groups]
        stat, p_value = stats.levene(*group_data)
        print(f'Levene\'s Test: W={stat:.4f}, p-value={p_value:.4f}')

# Check assumptions for RSSI within each power group
print("Checking assumptions for RSSI within each power group:")
check_assumptions(df, 'Material', 'RSSI', 'powerGroup')

# Check assumptions for RTT within each power group
print("\nChecking assumptions for RTT within each power group:")
check_assumptions(df, 'Material', 'RTT', 'powerGroup')

In [ ]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import statsmodels.api as sm
from statsmodels.formula.api import ols
from scipy.stats import f_oneway, ttest_ind
import itertools

# Sample data loading
df = pd.read_csv('your_data.csv')  # Replace with your actual data source

# List of unique power groups
power_groups = df['powerGroup'].unique()

# Perform ANOVA for each power group
anova_results = {}
for power in power_groups:
    subset = df[df['powerGroup'] == power]
    anova_model = ols('RSSI ~ C(Material)', data=subset).fit()
    anova_table = sm.stats.anova_lm(anova_model, typ=2)
    anova_results[power] = anova_table

# Calculate effect sizes (Cohen's d) between materials at each power level
def cohen_d(group1, group2):
    diff = group1.mean() - group2.mean()
    pooled_std = ((group1.std()**2 + group2.std()**2) / 2)**0.5
    return diff / pooled_std

effect_sizes = {}
for power in power_groups:
    subset = df[df['powerGroup'] == power]
    materials = subset['Material'].unique()
    combinations = list(itertools.combinations(materials, 2))
    for (mat1, mat2) in combinations:
        group1 = subset[subset['Material'] == mat1]['RSSI']
        group2 = subset[subset['Material'] == mat2]['RSSI']
        effect_sizes[(power, mat1, mat2)] = cohen_d(group1, group2)

# Visualize the results using Plotly (dark theme)
fig = make_subplots(rows=1, cols=2, subplot_titles=("ANOVA Results", "Effect Sizes"))

# ANOVA plot
anova_data = []
for power, table in anova_results.items():
    anova_data.append(go.Bar(name=f'Power Group {power}', x=table.index, y=table['PR(>F)']))

fig.add_traces(anova_data, rows=1, cols=1)

# Effect sizes plot
effect_data = []
for (power, mat1, mat2), effect_size in effect_sizes.items():
    effect_data.append(go.Bar(name=f'{power} ({mat1} vs {mat2})', x=[f'{mat1} vs {mat2}'], y=[effect_size]))

fig.add_traces(effect_data, rows=1, cols=2)

# Update layout
fig.update_layout(title_text="Statistical Analysis of Materials' Interaction with Signal Strength", template="plotly_dark")
fig.show()
